# Padchest Unimodal

In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import ast
from utils import *
from sklearn import preprocessing, metrics
import json
from keras.backend import squeeze, expand_dims
from sklearn.metrics import roc_auc_score, classification_report
from keras.models import load_model
from random import shuffle
from keras import backend as K

Using TensorFlow backend.


In [2]:
#Pipeline Parameters
createEmbeddings = False
createFolds = True
workers = 3
n_epochs = 80
image_loc = '/gpfs/ysm/pi/krauthammer/kl533/Indiana_University_Chest_X-ray_Collection'
fitVerbose = 0
with open('n_splits.json') as json_file:
    json_data = json.load(json_file)
n_splits = json_data['n_splits']
channels=3
shape=(256,256)
crop=(224,224)

In [3]:
#word2vec parameters
w2v_params = {'vocabulary_size':10000, 'window_size':5,'filters':'','vector_dim':224}
#parameters
max_text_len = 200#this is the same size as the sequence length
vocabulary_size = 10000
embedding_output_dim = crop[0]#whatever the image size is

In [4]:
#build our clean csv
#load already setup csv, with clean labels
suffix = '.png'
df = load_indiana_CXR(path=image_loc)
#if label is normal 1 else 0
normal_mask = df.labels.apply(lambda x: 1 if 'normal' in x else 0)
df['normal'] = normal_mask
#add correct paths
df['img_id'] = df['img_id'].apply(lambda x: image_loc + '/images/' + x + suffix)

In [5]:
num_class = len(set(normal_mask))

In [6]:
if createEmbeddings:
    print('Section 0.5: create embeddings')
    from w2v_keras import w2v_keras
    w2v = w2v_keras(**w2v_params)
    w2v.fit(df['radiology_report'], epochs=20)
    w2v.save_embeddings('w2v_embeddings.json')

## Build our tokenizer to pass into generators

In [7]:
tokenizer = Tokenizer(num_words = vocabulary_size, lower=True, filters='')#make this a param
tokenizer.fit_on_texts(filter_sentences(df['radiology_report'].tolist()))

filtering sentences
filtering sents and removing stopwords


## Build our one hot encoder so we have a uniform encoding across all data

In [8]:
ohe = preprocessing.OneHotEncoder(categories='auto')
ohe.fit(df['normal'].to_numpy().reshape(-1, 1))

OneHotEncoder(categorical_features=None, categories='auto', drop=None,
              dtype=<class 'numpy.float64'>, handle_unknown='error',
              n_values=None, sparse=True)

### Load Embeddings from JSON file and map tokens to embedding

In [9]:
#load embedding weights from file
location = 'w2v_embeddings.json'
import json
with open(location) as f:
    embs = json.load(f)
    emb = json.loads(embs)
embedding_weights = mapTokensToEmbedding(emb, tokenizer.word_index, 
                                         vocabulary_size)

In [10]:
from keras_applications.resnet_v2 import ResNet50V2
from keras_applications.densenet import DenseNet121
from keras.layers import GlobalAveragePooling2D, Dense
from keras import Sequential
def DenseNetModelFactory():
    model = DenseNet121(input_shape=(*crop,channels), include_top=False, weights=None,backend=keras.backend,
        layers=keras.layers,
        models=keras.models,
        utils=keras.utils)
    return model

In [11]:
from keras.models import Model
from keras.layers import Dense, Reshape, concatenate, Lambda, Average, Maximum, Add, Multiply, Concatenate, BatchNormalization, Activation, GlobalAveragePooling2D, GlobalAveragePooling1D, Flatten
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from keras.layers.embeddings import Embedding
from keras.engine.input_layer import Input
from keras.optimizers import Adam
from keras.activations import relu


def Generate_Model(model_type = 'LateFusion'):
    #set up our text input
    if model_type == 'LateFusion':
        input_text = Input(shape=(max_text_len,), name='text_input')
        embedding = Embedding(input_length=max_text_len, input_dim=vocabulary_size, output_dim=embedding_output_dim, 
                               name='embedding_layer', trainable=False, weights=[embedding_weights])
        text_embedding = embedding(input_text)
        #set up our image branch
        image_input = Input((*crop,channels), name='image_input')
        image_branch = DenseNetModelFactory()(image_input)
        image_branch = Flatten()(image_branch)
        image_branch = Dense(1024, activation='relu', name='dense_layer1_image_branch')(image_branch)
        image_branch = BatchNormalization()(image_branch)
        image_branch = Dropout(0.5)(image_branch)        
        image_branch = Dense(512, activation='relu', name='dense_layer2_image_branch')(image_branch)
        image_branch = BatchNormalization()(image_branch)
        image_branch = Dropout(0.5)(image_branch)
        image_branch_out = Dense(num_class, activation='sigmoid', name='image_branch_output')(image_branch)
        image_branch_model = Model(inputs=[image_input], outputs=[image_branch_out])
        #set up text branch
        text_branch = get_text_branch()(text_embedding)
        text_branch = Dense(1024, activation='relu', name='dense_layer1_text_branch')(text_branch)
        text_branch = BatchNormalization()(text_branch)
        text_branch = Dropout(0.5)(text_branch)        
        text_branch = Dense(512, activation='relu', name='dense_layer2_text_branch')(text_branch)
        text_branch = BatchNormalization()(text_branch)
        text_branch = Dropout(0.5)(text_branch)
        text_branch_out = Dense(num_class, activation='softmax', name='text_branch_output')(text_branch)
        text_branch_model = Model(inputs=[input_text], outputs=[text_branch_out])
        outputs = [model.outputs[0] for model in [text_branch_model, image_branch_model]]
        output = Average()(outputs)
        classificationModel = Model(inputs=[text_branch_model.input, image_branch_model.input], outputs=[output])
    if model_type == "EarlyFusion":
        input_text = Input(shape=(max_text_len,), name='text_input')
        embedding = Embedding(input_length=max_text_len, input_dim=vocabulary_size, output_dim=embedding_output_dim, 
                               name='embedding_layer', trainable=False, weights=[embedding_weights])
        embedded_text = embedding(input_text)
        text_embedding = Lambda(embedding_3Ch)(embedded_text)
        #text_embedding = embedded_text
        #set up our image input
        image_input = Input((*crop,channels), name='image_input')
        '''merge our data for early fusion. Essentially we are just concatenating our data together'''
        if channels == 1:
            expandedText = Lambda(lambda x: expand_dims(x, axis=-1))
            text_embedding = expandedText(text_embedding)
        merged = concatenate([text_embedding, image_input], axis=1, name='merged')
        x = DenseNetModelFactory()(merged)
        x = Flatten()(x)
        x = Dense(1024, activation='relu', name='dense_layer1_earlyFusion')(x)
        x = BatchNormalization()(x)
        x = Dropout(0.5)(x)        
        x = Dense(512, activation='relu', name='dense_layer2_earlyFusion')(x)
        x = BatchNormalization()(x)
        x = Dropout(0.5)(x)
        output = Dense(num_class, activation='softmax', name='output')(x)
        classificationModel = Model(inputs=[input_text, image_input], outputs=[output])
    if model_type == "EarlyFusion_Wide":
        input_text = Input(shape=(max_text_len,), name='text_input')
        embedding = Embedding(input_length=max_text_len, input_dim=vocabulary_size, output_dim=embedding_output_dim, 
                               name='embedding_layer', trainable=False, weights=[embedding_weights])
        embedded_text = embedding(input_text)
        text_embedding = embedded_text
        #set up our image input
        image_input = Input((*crop,channels), name='image_input')
        image_input_reshaped = Reshape((crop[0]*3,crop[0]))(image_input)#672 is because we have a 224x224 image with 3 channels, 224x3 channels = 627
        '''merge our data for early fusion. Essentially we are just concatenating our data together'''
        merged = concatenate([text_embedding, image_input_reshaped], axis=1, name='merged')
        x = get_text_branch()(merged)
        x = Dense(1024, activation='relu', name='dense_layer1')(x)
        x = BatchNormalization()(x)
        x = Dropout(0.5)(x)        
        x = Dense(512, activation='relu', name='dense_layer2')(x)
        x = BatchNormalization()(x)
        x = Dropout(0.5)(x)
        output = Dense(num_class, activation='softmax', name='output')(x)
        classificationModel = Model(inputs=[input_text, image_input], outputs=[output])        
    if model_type == 'ModelFusion':
        input_text = Input(shape=(max_text_len,), name='text_input')
        embedding = Embedding(input_length=max_text_len, input_dim=vocabulary_size, output_dim=embedding_output_dim, 
                               name='embedding_layer', trainable=False, weights=[embedding_weights])
        text_embedding = embedding(input_text)
        image_input = Input((*crop,channels), name='image_input')
        txt = get_text_branch()            
        #set up our image branch
        image_branch = DenseNetModelFactory()(image_input)
        image_branch = Flatten()(image_branch)
        #set up text branch
        text_branch = (txt)(text_embedding)
        #add some dense layers as test
        x = concatenate([image_branch, text_branch])
        x = Dense(1024, activation='relu', name='dense_layer1_modelFusion')(x)
        x = BatchNormalization()(x)
        x = Dropout(0.5)(x)        
        x = Dense(512, activation='relu', name='dense_layer2_modelFusion')(x)
        x = BatchNormalization()(x)
        x = Dropout(0.5)(x)
        #add some dense layers as test
        output = Dense(num_class, activation='softmax', name='output')(x)
        classificationModel = Model(inputs=[input_text, image_input], outputs=[output])
    if model_type == "text_1d":#text_1d only
        input_text = Input(shape=(max_text_len,), name='text_input')
        embedding = Embedding(input_length=max_text_len, input_dim=vocabulary_size, output_dim=embedding_output_dim, 
                               name='embedding_layer', trainable=False, weights=[embedding_weights])
        embedded_text = embedding(input_text)
        x = get_text_branch()(embedded_text)
        x = Dense(1024, activation='relu', name='dense_layer1_text_branch')(x)
        x = BatchNormalization()(x)
        x = Dropout(0.5)(x)        
        x = Dense(512, activation='relu', name='dense_layer2_text_branch')(x)
        x = BatchNormalization()(x)
        x = Dropout(0.5)(x)
        output = Dense(num_class, activation='softmax', name='text_output')(x)
        classificationModel = Model(inputs=[input_text], outputs=[output])
    if model_type == "image":#image only
        image_input = Input((*crop,channels), name='image_input')
        x = DenseNetModelFactory()(image_input)
        x = Flatten()(x)
        x = Dense(1024, activation='relu', name='dense_layer1_image_branch')(x)
        x = BatchNormalization()(x)
        x = Dropout(0.5)(x)        
        x = Dense(512, activation='relu', name='dense_layer2_image_branch')(x)
        x = BatchNormalization()(x)
        x = Dropout(0.5)(x)
        output = Dense(num_class, activation='softmax', name='image_output')(x)
        classificationModel = Model(inputs=[image_input], outputs=[output])
    return classificationModel

# Set up environment for Models

In [12]:
'''
Group by label, make each label its own 'database', split databases into n chunks, for each class append ith chunk
we now have chunks which contain stratified samples. Because we split databases based on labels we may
get some samples appearing in different chunks, but will never show up in valid or test
'''

#put all labels into their own chunk, each entry contains the splits for each label, so for label 0 
#we have nsplits of label 0 at location 0
from collections import defaultdict
labelChunks = {}
stratifiedChunks = defaultdict(list)
chunks = defaultdict(dict)
for i in range(num_class):
    labelGroup = list(set(df.iloc[df.groupby(['normal']).groups[i]].sample(frac=1.0)['sample_id'].tolist()))#get all samples of that label;shuffle data
    labelChunks[i] = np.array_split(labelGroup,n_splits)#get all labels
#for each split, get the ith chunk
for split in range(n_splits):
    for label, val in labelChunks.items():
        stratifiedChunks[split].extend(val[split].tolist())
#for each chunk save its sample_id
for i, (key, chunk) in enumerate(stratifiedChunks.items()):
    chunkSampleIDs = chunk#all sample ids
    #split sample ids so we dont get any data leakage
    shuffle(chunkSampleIDs)
    train, valid, test = np.split(chunkSampleIDs, [int(.8*len(chunkSampleIDs)), int(.9*len(chunkSampleIDs))])    
    chunks[i]['train'] = train.tolist()
    chunks[i]['valid'] = valid.tolist()
    chunks[i]['test'] = test.tolist()
with open('indianaFolds.json', 'w') as fp:
    json.dump(chunks, fp)

In [13]:
with open('indianaFolds.json') as f:
    data = json.load(f)#get all of the fold information

In [14]:
chunkList = []
for ix,chunk in data.items():
    chunkList.append(chunk['train'])
    chunkList.append(chunk['valid'])
    chunkList.append(chunk['test'])
for i in range(len(chunkList)):
    for j in range(i+1, len(chunkList)):
        assert len(set(chunkList[i]).intersection(chunkList[j])) == 0, "You have an element from one chunk in another! Each chunk should have unique indicies"

# Setup our Experiments

In [15]:
from keras.optimizers import Adam
import tensorflow as tf
def optimizerFactory():
    return Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, amsgrad=False)

In [16]:
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
def getCheckpointCallback(name):
    return ModelCheckpoint(name, monitor='acc', verbose=1, save_best_only=True, save_weights_only=False, mode='max', period=1)

# Train images

In [17]:
#scheduled learning rate
lrate = LearningRateScheduler(step_decay)

In [18]:
#Model statistics dataframe
model_stat_df = pd.DataFrame({}, columns=['model', 'f1-score', 'precision','recall','support','AUC'])

In [19]:
params = {'dim': (256,256),
          'crop': (224,224),
              'batch_size': 8,
              'n_channels': 3,
              'loc': image_loc,
              'shuffle': False,
              'modality': ['image'],
              'tokenizer':tokenizer,
               'ohe': ohe}
AUCList = []
report_list = []
for ix, split in data.items():
    K.clear_session()
    print('====================Starting Split Number {}===================='.format(ix))
    model = Generate_Model(model_type='image')#create model
    model.compile(optimizer=optimizerFactory(), loss='categorical_crossentropy', metrics=['accuracy'])
    
    train_generator = indianaDataGenerator_DF(df[df['sample_id'].isin(data[str(ix)]['train'])].reset_index(drop=True), **params)
    valid_generator = indianaDataGenerator_DF(df[df['sample_id'].isin(data[str(ix)]['valid'])].reset_index(drop=True), **params)
    test_generator = indianaDataGenerator_DF(df[df['sample_id'].isin(data[str(ix)]['test'])].reset_index(drop=True), **params)
    
    #checkpoint
    filename = 'imageOnly_split_{}.hd5'.format(ix)
    checkpoint = getCheckpointCallback(filename)
    model.fit_generator(train_generator, steps_per_epoch=len(train_generator), validation_data=valid_generator, 
                        validation_steps=len(valid_generator), workers=workers, epochs=n_epochs, verbose=fitVerbose, 
                        callbacks=[checkpoint,lrate])
    y_hat = model.predict_generator(test_generator, steps=len(test_generator), workers=workers)
    y_hat_class=y_hat.argmax(-1)
    y_true = ohe.transform(np.array(df[df['sample_id'].isin(data[str(ix)]['test'])]['normal']).reshape(-1, 1)).toarray()
    y_true_class = y_true.argmax(-1)
    #classification report
    cr = classification_report(y_true_class[:len(y_hat)], y_hat_class, target_names=None, output_dict=True)
    report_list.append(pd.DataFrame(cr).transpose())
    #area under curve
    auc = roc_auc_score(y_true[:len(y_hat)], y_hat)
    AUCList.append(auc)
    print('AUC score for split {}: {}'.format(ix,auc))

classification_report(y_true_class[:len(y_hat)], y_hat_class)
model_stat_df = model_stat_df.append({'model':"Image only", 'f1-score': df_mean(report_list)['f1-score'].loc['weighted avg'], 'precision':  df_mean(report_list)['precision'].loc['weighted avg'],
                                     'recall': df_mean(report_list)['recall'].loc['weighted avg'], 'support': df_mean(report_list)['support'].loc['weighted avg'], 'AUC': np.mean(AUCList)},ignore_index=True)

W0224 12:01:30.066973 47685665518464 deprecation_wrapper.py:119] From /gpfs/ysm/project/kl533/conda_envs/dlnn/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:95: The name tf.reset_default_graph is deprecated. Please use tf.compat.v1.reset_default_graph instead.

W0224 12:01:30.068329 47685665518464 deprecation_wrapper.py:119] From /gpfs/ysm/project/kl533/conda_envs/dlnn/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:98: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0224 12:01:30.209666 47685665518464 deprecation_wrapper.py:119] From /gpfs/ysm/project/kl533/conda_envs/dlnn/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:102: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0224 12:01:30.211053 47685665518464 deprecation_wrapper.py:119] From /gpfs/ysm/project/kl533/conda_envs/dlnn/lib/python3.6/site-packages/keras/backend/tens

====================Starting Split Number 0====================


W0224 12:01:45.146029 47685665518464 deprecation_wrapper.py:119] From /gpfs/ysm/project/kl533/conda_envs/dlnn/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:1834: The name tf.nn.fused_batch_norm is deprecated. Please use tf.compat.v1.nn.fused_batch_norm instead.

W0224 12:01:45.255411 47685665518464 deprecation_wrapper.py:119] From /gpfs/ysm/project/kl533/conda_envs/dlnn/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0224 12:01:46.235662 47685665518464 deprecation_wrapper.py:119] From /gpfs/ysm/project/kl533/conda_envs/dlnn/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3980: The name tf.nn.avg_pool is deprecated. Please use tf.nn.avg_pool2d instead.

W0224 12:02:01.071271 47685665518464 deprecation.py:506] From /gpfs/ysm/project/kl533/conda_envs/dlnn/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.

Lowering LR to 0.0001

Epoch 00001: acc improved from -inf to 0.53464, saving model to imageOnly_split_0.hd5
Lowering LR to 0.0001

Epoch 00002: acc improved from 0.53464 to 0.56342, saving model to imageOnly_split_0.hd5
Lowering LR to 0.0001

Epoch 00003: acc did not improve from 0.56342
Lowering LR to 0.0001

Epoch 00004: acc improved from 0.56342 to 0.57279, saving model to imageOnly_split_0.hd5
Lowering LR to 0.0001

Epoch 00005: acc improved from 0.57279 to 0.57831, saving model to imageOnly_split_0.hd5
Lowering LR to 0.0001

Epoch 00006: acc improved from 0.57831 to 0.58066, saving model to imageOnly_split_0.hd5
Lowering LR to 0.0001

Epoch 00007: acc improved from 0.58066 to 0.58768, saving model to imageOnly_split_0.hd5
Lowering LR to 0.0001

Epoch 00008: acc improved from 0.58768 to 0.60023, saving model to imageOnly_split_0.hd5
Lowering LR to 0.0001

Epoch 00009: acc did not improve from 0.60023
Lowering LR to 5e-05

Epoch 00010: acc improved from 0.60023 to 0.62216, saving m

## Text only

In [20]:
params = {'dim': (256,256),
          'crop': (224,224),
              'batch_size': 8,
              'n_channels': 3,
              'loc': image_loc,
              'shuffle': False,
              'modality': ['text'],
              'tokenizer':tokenizer,
               'ohe': ohe}
AUCList = []
report_list = []
for ix, split in data.items():
    K.clear_session()
    print('====================Starting Split Number {}===================='.format(ix))
    model = Generate_Model(model_type='text_1d')#create model
    model.compile(optimizer=optimizerFactory(), loss='categorical_crossentropy', metrics=['accuracy'])
    
    train_generator = indianaDataGenerator_DF(df[df['sample_id'].isin(data[str(ix)]['train'])].reset_index(drop=True), **params)
    valid_generator = indianaDataGenerator_DF(df[df['sample_id'].isin(data[str(ix)]['valid'])].reset_index(drop=True), **params)
    test_generator = indianaDataGenerator_DF(df[df['sample_id'].isin(data[str(ix)]['test'])].reset_index(drop=True), **params)
    
    #checkpoint
    filename = 'textOnly_split_{}.hd5'.format(ix)
    checkpoint = getCheckpointCallback(filename)
    model.fit_generator(train_generator, steps_per_epoch=len(train_generator), validation_data=valid_generator, 
                        validation_steps=len(valid_generator), workers=workers, epochs=n_epochs, verbose=fitVerbose, 
                        callbacks=[checkpoint,lrate])
    y_hat = model.predict_generator(test_generator, steps=len(test_generator), workers=workers)
    y_hat_class=y_hat.argmax(-1)
    y_true = ohe.transform(np.array(df[df['sample_id'].isin(data[str(ix)]['test'])]['normal']).reshape(-1, 1)).toarray()
    y_true_class = y_true.argmax(-1)
    #classification report
    cr = classification_report(y_true_class[:len(y_hat)], y_hat_class, target_names=None, output_dict=True)
    report_list.append(pd.DataFrame(cr).transpose())
    #area under curve
    auc = roc_auc_score(y_true[:len(y_hat)], y_hat)
    AUCList.append(auc)
    print('AUC score for split {}: {}'.format(ix,auc))

classification_report(y_true_class[:len(y_hat)], y_hat_class)
model_stat_df = model_stat_df.append({'model':"Text only (Wide)", 'f1-score': df_mean(report_list)['f1-score'].loc['weighted avg'], 'precision':  df_mean(report_list)['precision'].loc['weighted avg'],
                                     'recall': df_mean(report_list)['recall'].loc['weighted avg'], 'support': df_mean(report_list)['support'].loc['weighted avg'], 'AUC': np.mean(AUCList)},ignore_index=True)

====================Starting Split Number 0====================
You have passed in a tokenizer, we will just convert to sequences
filtering sentences
filtering sents and removing stopwords
You have passed in a tokenizer, we will just convert to sequences
filtering sentences
filtering sents and removing stopwords
You have passed in a tokenizer, we will just convert to sequences
filtering sentences
filtering sents and removing stopwords
Lowering LR to 0.0001

Epoch 00001: acc improved from -inf to 0.63621, saving model to textOnly_split_0.hd5
Lowering LR to 0.0001

Epoch 00002: acc improved from 0.63621 to 0.70398, saving model to textOnly_split_0.hd5
Lowering LR to 0.0001

Epoch 00003: acc improved from 0.70398 to 0.73996, saving model to textOnly_split_0.hd5
Lowering LR to 0.0001

Epoch 00004: acc improved from 0.73996 to 0.76790, saving model to textOnly_split_0.hd5
Lowering LR to 0.0001

Epoch 00005: acc improved from 0.76790 to 0.79501, saving model to textOnly_split_0.hd5
Lowering 

# Train multimodal Models

In [21]:
params = {'dim': (256,256),
          'crop': (224,224),
              'batch_size': 8,
              'n_channels': 3,
              'loc': image_loc,
              'shuffle': False,
              'modality': ['image', 'text'],
              'tokenizer':tokenizer,
               'ohe': ohe}

## Model Fusion Randomly Initialized

In [22]:
AUCList = []
report_list = []
for ix, split in data.items():
    K.clear_session()
    print('====================Starting Split Number {}===================='.format(ix))
    model = Generate_Model(model_type='ModelFusion')#create model
    model.compile(optimizer=optimizerFactory(), loss='categorical_crossentropy', metrics=['accuracy'])
    
    train_generator = indianaDataGenerator_DF(df[df['sample_id'].isin(data[str(ix)]['train'])].reset_index(drop=True), **params)
    valid_generator = indianaDataGenerator_DF(df[df['sample_id'].isin(data[str(ix)]['valid'])].reset_index(drop=True), **params)
    test_generator = indianaDataGenerator_DF(df[df['sample_id'].isin(data[str(ix)]['test'])].reset_index(drop=True), **params)
    
    #checkpoint
    filename = 'modelFusion_split_{}.hd5'.format(ix)
    checkpoint = getCheckpointCallback(filename)
    model.fit_generator(train_generator, steps_per_epoch=len(train_generator), validation_data=valid_generator, 
                        validation_steps=len(valid_generator), workers=workers, epochs=n_epochs, verbose=fitVerbose, 
                        callbacks=[checkpoint,lrate])
    y_hat = model.predict_generator(test_generator, steps=len(test_generator), workers=workers)
    y_hat_class=y_hat.argmax(-1)
    y_true = ohe.transform(np.array(df[df['sample_id'].isin(data[str(ix)]['test'])]['normal']).reshape(-1, 1)).toarray()
    y_true_class = y_true.argmax(-1)
    #classification report
    cr = classification_report(y_true_class[:len(y_hat)], y_hat_class, target_names=None, output_dict=True)
    report_list.append(pd.DataFrame(cr).transpose())
    #area under curve
    auc = roc_auc_score(y_true[:len(y_hat)], y_hat)
    AUCList.append(auc)
    print('AUC score for split {}: {}'.format(ix,auc))

classification_report(y_true_class[:len(y_hat)], y_hat_class)
model_stat_df = model_stat_df.append({'model':"ModelFusion", 'f1-score': df_mean(report_list)['f1-score'].loc['weighted avg'], 'precision':  df_mean(report_list)['precision'].loc['weighted avg'],
                                     'recall': df_mean(report_list)['recall'].loc['weighted avg'], 'support': df_mean(report_list)['support'].loc['weighted avg'], 'AUC': np.mean(AUCList)},ignore_index=True)

====================Starting Split Number 0====================
You have passed in a tokenizer, we will just convert to sequences
filtering sentences
filtering sents and removing stopwords
You have passed in a tokenizer, we will just convert to sequences
filtering sentences
filtering sents and removing stopwords
You have passed in a tokenizer, we will just convert to sequences
filtering sentences
filtering sents and removing stopwords
Lowering LR to 0.0001

Epoch 00001: acc improved from -inf to 0.60007, saving model to modelFusion_split_0.hd5
Lowering LR to 0.0001

Epoch 00002: acc improved from 0.60007 to 0.69244, saving model to modelFusion_split_0.hd5
Lowering LR to 0.0001

Epoch 00003: acc improved from 0.69244 to 0.77108, saving model to modelFusion_split_0.hd5
Lowering LR to 0.0001

Epoch 00004: acc improved from 0.77108 to 0.80723, saving model to modelFusion_split_0.hd5
Lowering LR to 0.0001

Epoch 00005: acc improved from 0.80723 to 0.83250, saving model to modelFusion_split_

## Model Fusion Pretrained

In [23]:
AUCList = []
report_list = []
for ix, split in data.items():
    K.clear_session()
    print('====================Starting Split Number {}===================='.format(ix))
    model = Generate_Model(model_type='ModelFusion')#create model
    model.compile(optimizer=optimizerFactory(), loss='categorical_crossentropy', metrics=['accuracy'])
    
    train_generator = indianaDataGenerator_DF(df[df['sample_id'].isin(data[str(ix)]['train'])].reset_index(drop=True), **params)
    valid_generator = indianaDataGenerator_DF(df[df['sample_id'].isin(data[str(ix)]['valid'])].reset_index(drop=True), **params)
    test_generator = indianaDataGenerator_DF(df[df['sample_id'].isin(data[str(ix)]['test'])].reset_index(drop=True), **params)
    
    #checkpoint
    filename = 'modelFusion_PT_split_{}.hd5'.format(ix)
    checkpoint = getCheckpointCallback(filename)
    
    #Load weights
    model.layers[2].set_weights(load_model('imageOnly_split_{}.hd5'.format(ix),compile=False).layers[1].get_weights())#image
    model.layers[5].set_weights(load_model('textOnly_split_{}.hd5'.format(ix),compile=False).layers[2].get_weights())#text
    
    model.fit_generator(train_generator, steps_per_epoch=len(train_generator), validation_data=valid_generator, 
                        validation_steps=len(valid_generator), workers=workers, epochs=n_epochs, verbose=fitVerbose, 
                        callbacks=[checkpoint,lrate])
    y_hat = model.predict_generator(test_generator, steps=len(test_generator), workers=workers)
    y_hat_class=y_hat.argmax(-1)
    y_true = ohe.transform(np.array(df[df['sample_id'].isin(data[str(ix)]['test'])]['normal']).reshape(-1, 1)).toarray()
    y_true_class = y_true.argmax(-1)
    #classification report
    cr = classification_report(y_true_class[:len(y_hat)], y_hat_class, target_names=None, output_dict=True)
    report_list.append(pd.DataFrame(cr).transpose())
    #area under curve
    auc = roc_auc_score(y_true[:len(y_hat)], y_hat)
    AUCList.append(auc)
    print('AUC score for split {}: {}'.format(ix,auc))

classification_report(y_true_class[:len(y_hat)], y_hat_class)
model_stat_df = model_stat_df.append({'model':"ModelFusion_PT", 'f1-score': df_mean(report_list)['f1-score'].loc['weighted avg'], 'precision':  df_mean(report_list)['precision'].loc['weighted avg'],
                                     'recall': df_mean(report_list)['recall'].loc['weighted avg'], 'support': df_mean(report_list)['support'].loc['weighted avg'], 'AUC': np.mean(AUCList)},ignore_index=True)

====================Starting Split Number 0====================
You have passed in a tokenizer, we will just convert to sequences
filtering sentences
filtering sents and removing stopwords
You have passed in a tokenizer, we will just convert to sequences
filtering sentences
filtering sents and removing stopwords
You have passed in a tokenizer, we will just convert to sequences
filtering sentences
filtering sents and removing stopwords
Lowering LR to 0.0001

Epoch 00001: acc improved from -inf to 0.76807, saving model to modelFusion_PT_split_0.hd5
Lowering LR to 0.0001

Epoch 00002: acc improved from 0.76807 to 0.81844, saving model to modelFusion_PT_split_0.hd5
Lowering LR to 0.0001

Epoch 00003: acc improved from 0.81844 to 0.83300, saving model to modelFusion_PT_split_0.hd5
Lowering LR to 0.0001

Epoch 00004: acc improved from 0.83300 to 0.86094, saving model to modelFusion_PT_split_0.hd5
Lowering LR to 0.0001

Epoch 00005: acc improved from 0.86094 to 0.87718, saving model to modelF

## Late Fusion Randomly Initialized

In [24]:
AUCList = []
report_list = []
for ix, split in data.items():
    K.clear_session()
    print('====================Starting Split Number {}===================='.format(ix))
    model = Generate_Model(model_type='LateFusion')#create model
    model.compile(optimizer=optimizerFactory(), loss='categorical_crossentropy', metrics=['accuracy'])
    
    train_generator = indianaDataGenerator_DF(df[df['sample_id'].isin(data[str(ix)]['train'])].reset_index(drop=True), **params)
    valid_generator = indianaDataGenerator_DF(df[df['sample_id'].isin(data[str(ix)]['valid'])].reset_index(drop=True), **params)
    test_generator = indianaDataGenerator_DF(df[df['sample_id'].isin(data[str(ix)]['test'])].reset_index(drop=True), **params)
    
    #checkpoint
    filename = 'lateFusion_split_{}.hd5'.format(ix)
    checkpoint = getCheckpointCallback(filename)
    
    model.fit_generator(train_generator, steps_per_epoch=len(train_generator), validation_data=valid_generator, 
                        validation_steps=len(valid_generator), workers=workers, epochs=n_epochs, verbose=fitVerbose, 
                        callbacks=[checkpoint,lrate])
    y_hat = model.predict_generator(test_generator, steps=len(test_generator), workers=workers)
    y_hat_class=y_hat.argmax(-1)
    y_true = ohe.transform(np.array(df[df['sample_id'].isin(data[str(ix)]['test'])]['normal']).reshape(-1, 1)).toarray()
    y_true_class = y_true.argmax(-1)
    #classification report
    cr = classification_report(y_true_class[:len(y_hat)], y_hat_class, target_names=None, output_dict=True)
    report_list.append(pd.DataFrame(cr).transpose())
    #area under curve
    auc = roc_auc_score(y_true[:len(y_hat)], y_hat)
    AUCList.append(auc)
    print('AUC score for split {}: {}'.format(ix,auc))

classification_report(y_true_class[:len(y_hat)], y_hat_class)
model_stat_df = model_stat_df.append({'model':"LateFusion", 'f1-score': df_mean(report_list)['f1-score'].loc['weighted avg'], 'precision':  df_mean(report_list)['precision'].loc['weighted avg'],
                                     'recall': df_mean(report_list)['recall'].loc['weighted avg'], 'support': df_mean(report_list)['support'].loc['weighted avg'], 'AUC': np.mean(AUCList)},ignore_index=True)

====================Starting Split Number 0====================
You have passed in a tokenizer, we will just convert to sequences
filtering sentences
filtering sents and removing stopwords
You have passed in a tokenizer, we will just convert to sequences
filtering sentences
filtering sents and removing stopwords
You have passed in a tokenizer, we will just convert to sequences
filtering sentences
filtering sents and removing stopwords
Lowering LR to 0.0001

Epoch 00001: acc improved from -inf to 0.67654, saving model to lateFusion_split_0.hd5
Lowering LR to 0.0001

Epoch 00002: acc improved from 0.67654 to 0.75469, saving model to lateFusion_split_0.hd5
Lowering LR to 0.0001

Epoch 00003: acc improved from 0.75469 to 0.77393, saving model to lateFusion_split_0.hd5
Lowering LR to 0.0001

Epoch 00004: acc improved from 0.77393 to 0.80003, saving model to lateFusion_split_0.hd5
Lowering LR to 0.0001

Epoch 00005: acc improved from 0.80003 to 0.81928, saving model to lateFusion_split_0.hd5

## Late Fusion Pretrained

In [25]:
AUCList = []
report_list = []
for ix, split in data.items():
    K.clear_session()
    print('====================Starting Split Number {}===================='.format(ix))
    model = Generate_Model(model_type='LateFusion')#create model
    model.compile(optimizer=optimizerFactory(), loss='categorical_crossentropy', metrics=['accuracy'])
    
    train_generator = indianaDataGenerator_DF(df[df['sample_id'].isin(data[str(ix)]['train'])].reset_index(drop=True), **params)
    valid_generator = indianaDataGenerator_DF(df[df['sample_id'].isin(data[str(ix)]['valid'])].reset_index(drop=True), **params)
    test_generator = indianaDataGenerator_DF(df[df['sample_id'].isin(data[str(ix)]['test'])].reset_index(drop=True), **params)
    
    #checkpoint
    filename = 'lateFusion_PT_split_{}.hd5'.format(ix)
    checkpoint = getCheckpointCallback(filename)
    #Load weights: This will only load the feature extractors, not the 2 dense layers at the end.
    model.layers[3].set_weights(load_model('imageOnly_split_{}.hd5'.format(ix),compile=False).layers[1].get_weights())#image
    model.layers[4].set_weights(load_model('textOnly_split_{}.hd5'.format(ix),compile=False).layers[2].get_weights())#text
    
    model.fit_generator(train_generator, steps_per_epoch=len(train_generator), validation_data=valid_generator, 
                        validation_steps=len(valid_generator), workers=workers, epochs=n_epochs, verbose=fitVerbose, 
                        callbacks=[checkpoint,lrate])
    y_hat = model.predict_generator(test_generator, steps=len(test_generator), workers=workers)
    y_hat_class=y_hat.argmax(-1)
    y_true = ohe.transform(np.array(df[df['sample_id'].isin(data[str(ix)]['test'])]['normal']).reshape(-1, 1)).toarray()
    y_true_class = y_true.argmax(-1)
    #classification report
    cr = classification_report(y_true_class[:len(y_hat)], y_hat_class, target_names=None, output_dict=True)
    report_list.append(pd.DataFrame(cr).transpose())
    #area under curve
    auc = roc_auc_score(y_true[:len(y_hat)], y_hat)
    AUCList.append(auc)
    print('AUC score for split {}: {}'.format(ix,auc))

classification_report(y_true_class[:len(y_hat)], y_hat_class)
model_stat_df = model_stat_df.append({'model':"LateFusion_PT", 'f1-score': df_mean(report_list)['f1-score'].loc['weighted avg'], 'precision':  df_mean(report_list)['precision'].loc['weighted avg'],
                                     'recall': df_mean(report_list)['recall'].loc['weighted avg'], 'support': df_mean(report_list)['support'].loc['weighted avg'], 'AUC': np.mean(AUCList)},ignore_index=True)

====================Starting Split Number 0====================
You have passed in a tokenizer, we will just convert to sequences
filtering sentences
filtering sents and removing stopwords
You have passed in a tokenizer, we will just convert to sequences
filtering sentences
filtering sents and removing stopwords
You have passed in a tokenizer, we will just convert to sequences
filtering sentences
filtering sents and removing stopwords
Lowering LR to 0.0001

Epoch 00001: acc improved from -inf to 0.82915, saving model to lateFusion_PT_split_0.hd5
Lowering LR to 0.0001

Epoch 00002: acc improved from 0.82915 to 0.86981, saving model to lateFusion_PT_split_0.hd5
Lowering LR to 0.0001

Epoch 00003: acc improved from 0.86981 to 0.88069, saving model to lateFusion_PT_split_0.hd5
Lowering LR to 0.0001

Epoch 00004: acc improved from 0.88069 to 0.89709, saving model to lateFusion_PT_split_0.hd5
Lowering LR to 0.0001

Epoch 00005: acc improved from 0.89709 to 0.90060, saving model to lateFusion

## Early Fusion

In [26]:
AUCList = []
report_list = []
for ix, split in data.items():
    K.clear_session()
    print('====================Starting Split Number {}===================='.format(ix))
    model = Generate_Model(model_type='EarlyFusion')#create model
    model.compile(optimizer=optimizerFactory(), loss='categorical_crossentropy', metrics=['accuracy'])
    
    train_generator = indianaDataGenerator_DF(df[df['sample_id'].isin(data[str(ix)]['train'])].reset_index(drop=True), **params)
    valid_generator = indianaDataGenerator_DF(df[df['sample_id'].isin(data[str(ix)]['valid'])].reset_index(drop=True), **params)
    test_generator = indianaDataGenerator_DF(df[df['sample_id'].isin(data[str(ix)]['test'])].reset_index(drop=True), **params)
    
    #checkpoint
    filename = 'earlyFusion_split_{}.hd5'.format(ix)
    checkpoint = getCheckpointCallback(filename)
    
    model.fit_generator(train_generator, steps_per_epoch=len(train_generator), validation_data=valid_generator, 
                        validation_steps=len(valid_generator), workers=workers, epochs=n_epochs, verbose=fitVerbose, 
                        callbacks=[checkpoint,lrate])
    y_hat = model.predict_generator(test_generator, steps=len(test_generator), workers=workers)
    y_hat_class=y_hat.argmax(-1)
    y_true = ohe.transform(np.array(df[df['sample_id'].isin(data[str(ix)]['test'])]['normal']).reshape(-1, 1)).toarray()
    y_true_class = y_true.argmax(-1)
    #classification report
    cr = classification_report(y_true_class[:len(y_hat)], y_hat_class, target_names=None, output_dict=True)
    report_list.append(pd.DataFrame(cr).transpose())
    #area under curve
    auc = roc_auc_score(y_true[:len(y_hat)], y_hat)
    AUCList.append(auc)
    print('AUC score for split {}: {}'.format(ix,auc))

classification_report(y_true_class[:len(y_hat)], y_hat_class)
model_stat_df = model_stat_df.append({'model':"EarlyFusion", 'f1-score': df_mean(report_list)['f1-score'].loc['weighted avg'], 'precision':  df_mean(report_list)['precision'].loc['weighted avg'],
                                     'recall': df_mean(report_list)['recall'].loc['weighted avg'], 'support': df_mean(report_list)['support'].loc['weighted avg'], 'AUC': np.mean(AUCList)},ignore_index=True)

====================Starting Split Number 0====================
You have passed in a tokenizer, we will just convert to sequences
filtering sentences
filtering sents and removing stopwords
You have passed in a tokenizer, we will just convert to sequences
filtering sentences
filtering sents and removing stopwords
You have passed in a tokenizer, we will just convert to sequences
filtering sentences
filtering sents and removing stopwords
Lowering LR to 0.0001

Epoch 00001: acc improved from -inf to 0.62433, saving model to earlyFusion_split_0.hd5
Lowering LR to 0.0001

Epoch 00002: acc improved from 0.62433 to 0.68307, saving model to earlyFusion_split_0.hd5
Lowering LR to 0.0001

Epoch 00003: acc improved from 0.68307 to 0.75870, saving model to earlyFusion_split_0.hd5
Lowering LR to 0.0001

Epoch 00004: acc improved from 0.75870 to 0.77845, saving model to earlyFusion_split_0.hd5
Lowering LR to 0.0001

Epoch 00005: acc improved from 0.77845 to 0.80371, saving model to earlyFusion_split_

## Early Fusion Wide Kernel

In [27]:
AUCList = []
report_list = []
for ix, split in data.items():
    K.clear_session()
    print('====================Starting Split Number {}===================='.format(ix))
    model = Generate_Model(model_type='EarlyFusion_Wide')#create model
    model.compile(optimizer=optimizerFactory(), loss='categorical_crossentropy', metrics=['accuracy'])
    
    train_generator = indianaDataGenerator_DF(df[df['sample_id'].isin(data[str(ix)]['train'])].reset_index(drop=True), **params)
    valid_generator = indianaDataGenerator_DF(df[df['sample_id'].isin(data[str(ix)]['valid'])].reset_index(drop=True), **params)
    test_generator = indianaDataGenerator_DF(df[df['sample_id'].isin(data[str(ix)]['test'])].reset_index(drop=True), **params)
    
    #checkpoint
    filename = 'earlyFusion_Wide_split_{}.hd5'.format(ix)
    checkpoint = getCheckpointCallback(filename)
    
    model.fit_generator(train_generator, steps_per_epoch=len(train_generator), validation_data=valid_generator, 
                        validation_steps=len(valid_generator), workers=workers, epochs=n_epochs, verbose=fitVerbose, 
                        callbacks=[checkpoint,lrate])
    y_hat = model.predict_generator(test_generator, steps=len(test_generator), workers=workers)
    y_hat_class=y_hat.argmax(-1)
    y_true = ohe.transform(np.array(df[df['sample_id'].isin(data[str(ix)]['test'])]['normal']).reshape(-1, 1)).toarray()
    y_true_class = y_true.argmax(-1)
    #classification report
    cr = classification_report(y_true_class[:len(y_hat)], y_hat_class, target_names=None, output_dict=True)
    report_list.append(pd.DataFrame(cr).transpose())
    #area under curve
    auc = roc_auc_score(y_true[:len(y_hat)], y_hat)
    AUCList.append(auc)
    print('AUC score for split {}: {}'.format(ix,auc))

classification_report(y_true_class[:len(y_hat)], y_hat_class)
model_stat_df = model_stat_df.append({'model':"EarlyFusion_Wide", 'f1-score': df_mean(report_list)['f1-score'].loc['weighted avg'], 'precision':  df_mean(report_list)['precision'].loc['weighted avg'],
                                     'recall': df_mean(report_list)['recall'].loc['weighted avg'], 'support': df_mean(report_list)['support'].loc['weighted avg'], 'AUC': np.mean(AUCList)},ignore_index=True)

====================Starting Split Number 0====================
You have passed in a tokenizer, we will just convert to sequences
filtering sentences
filtering sents and removing stopwords
You have passed in a tokenizer, we will just convert to sequences
filtering sentences
filtering sents and removing stopwords
You have passed in a tokenizer, we will just convert to sequences
filtering sentences
filtering sents and removing stopwords
Lowering LR to 0.0001

Epoch 00001: acc improved from -inf to 0.59388, saving model to earlyFusion_Wide_split_0.hd5
Lowering LR to 0.0001

Epoch 00002: acc improved from 0.59388 to 0.65177, saving model to earlyFusion_Wide_split_0.hd5
Lowering LR to 0.0001

Epoch 00003: acc improved from 0.65177 to 0.70348, saving model to earlyFusion_Wide_split_0.hd5
Lowering LR to 0.0001

Epoch 00004: acc improved from 0.70348 to 0.75268, saving model to earlyFusion_Wide_split_0.hd5
Lowering LR to 0.0001

Epoch 00005: acc improved from 0.75268 to 0.78029, saving model t

In [28]:
model_stat_df.to_csv(path_or_buf="stats.csv", index=False)